####  Source Code from : 
https://github.com/clovaai/bros/blob/55c52d0872ed61fb7586b70618f45dcb0354f1b2/preprocess/funsd_spade/preprocess.py#L74-L86

In [28]:
import json
import os
from glob import glob

import imagesize
from tqdm import tqdm
from transformers import BertTokenizer

MAX_SEQ_LENGTH = 512
MODEL_TYPE = "bert"
VOCA = "bert-base-uncased"

INPUT_PATH = "..\..\Datasets\dataset"

OUTPUT_PATH = "spade"

os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_PATH, "preprocessed"), exist_ok=True)


def main():
    tokenizer = BertTokenizer.from_pretrained(VOCA, do_lower_case=True)

    for dataset_split in ["train", "val"]:
        print(f"dataset_split: {dataset_split}")
        do_preprocess(tokenizer, dataset_split)

    os.system(f"cp -r {os.path.join(INPUT_PATH, 'training_data')} {OUTPUT_PATH}")
    os.system(f"cp -r {os.path.join(INPUT_PATH, 'testing_data')} {OUTPUT_PATH}")
    save_class_names()


def do_preprocess(tokenizer, dataset_split):
    
    if dataset_split == "train":
        dataset_root_path = os.path.join(INPUT_PATH, "training_data")
    elif dataset_split == "val":
        dataset_root_path = os.path.join(INPUT_PATH, "testing_data")
    else:
        raise ValueError(f"Invalid dataset_split={dataset_split}")

    json_files = glob(os.path.join(dataset_root_path, "annotations", "*.json"))
    print(os.path.join(dataset_root_path, "annotations", "*.json"))
    print(json_files)
    preprocessed_fnames = []
    for json_file in tqdm(json_files):
    
        in_json_obj = json.load(open(json_file, "r", encoding="utf-8"))

        out_json_obj = {}
        out_json_obj["words"] = []
        out_json_obj["parse"] = {}
        out_json_obj["parse"]["class"] = {"header": [], "question": [], "answer": []}
        out_json_obj["parse"]["relations"] = []

        form_id_to_word_idx = {}
        other_seq_list = {}

        # words
        for form_idx, form in enumerate(in_json_obj["form"]):
            print(form)
            form_id = form["id"]
            form_text = form["text"].strip()
            form_label = form["label"]
            form_linking = form["linking"]

            if len(form_text) == 0:
                continue

            word_cnt = 0
            class_seq = []
            for word_idx, word in enumerate(form["words"]):
                word_text = word["text"]
                bb = word["box"]
                bb = [[bb[0], bb[1]], [bb[2], bb[1]], [bb[2], bb[3]], [bb[0], bb[3]]]
                tokens = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(word_text))

                word_obj = {"text": word_text, "tokens": tokens, "boundingBox": bb}
                if len(word_text) != 0:
                    out_json_obj["words"].append(word_obj)
                    word_cnt += 1
                    class_seq.append(len(out_json_obj["words"]) - 1)

            is_valid_class = False if form_label == "other" else True
            if is_valid_class:
                out_json_obj["parse"]["class"][form_label].append(class_seq)
                form_id_to_word_idx[form_id] = len(out_json_obj["words"]) - word_cnt
            else:
                other_seq_list[form_id] = class_seq

        # parse
        for form_idx, form in enumerate(in_json_obj["form"]):
            form_id = form["id"]
            form_text = form["text"].strip()
            form_label = form["label"]
            form_linking = form["linking"]

            if len(form_linking) == 0:
                continue

            for link_idx, link in enumerate(form_linking):
                if link[0] == form_id:
                    if (
                        link[1] in form_id_to_word_idx
                        and link[0] in form_id_to_word_idx
                    ):
                        relation_pair = [
                            form_id_to_word_idx[link[0]],
                            form_id_to_word_idx[link[1]],
                        ]
                        out_json_obj["parse"]["relations"].append(relation_pair)

        # meta
        out_json_obj["meta"] = {}
        image_file = (
            os.path.splitext(json_file.replace("annotations", "images"))[0] + ".png"
        )
        if dataset_split == "train":
            out_json_obj["meta"]["image_path"] = image_file[
                image_file.find("training_data/") :
            ]
        elif dataset_split == "val":
            out_json_obj["meta"]["image_path"] = image_file[
                image_file.find("testing_data/") :
            ]
        width, height = imagesize.get(image_file)
        out_json_obj["meta"]["imageSize"] = {"width": width, "height": height}
        out_json_obj["meta"]["voca"] = VOCA

        this_file_name = os.path.basename(json_file)

        # Save file name to list
        preprocessed_fnames.append(os.path.join("preprocessed", this_file_name))

        # Save to file
        data_obj_file = os.path.join(OUTPUT_PATH, "preprocessed", this_file_name)
        with open(data_obj_file, "w", encoding="utf-8") as fp:
            json.dump(out_json_obj, fp, ensure_ascii=False)

    # Save file name list file
    preprocessed_filelist_file = os.path.join(
        OUTPUT_PATH, f"preprocessed_files_{dataset_split}.txt"
    )
    with open(preprocessed_filelist_file, "w", encoding="utf-8") as fp:
        fp.write("\n".join(preprocessed_fnames))


def save_class_names():
    with open(
        os.path.join(OUTPUT_PATH, "class_names.txt"), "w", encoding="utf-8"
    ) as fp:
        fp.write("\n".join(["others", "header", "question", "answer"]))



In [29]:
main()

dataset_split: train
..\..\Datasets\dataset\training_data\annotations\*.json
['..\\..\\Datasets\\dataset\\training_data\\annotations\\0000971160.json', '..\\..\\Datasets\\dataset\\training_data\\annotations\\0000989556.json', '..\\..\\Datasets\\dataset\\training_data\\annotations\\0000990274.json', '..\\..\\Datasets\\dataset\\training_data\\annotations\\0000999294.json', '..\\..\\Datasets\\dataset\\training_data\\annotations\\0001118259.json', '..\\..\\Datasets\\dataset\\training_data\\annotations\\0001123541.json', '..\\..\\Datasets\\dataset\\training_data\\annotations\\0001129658.json', '..\\..\\Datasets\\dataset\\training_data\\annotations\\0001209043.json', '..\\..\\Datasets\\dataset\\training_data\\annotations\\0001239897.json', '..\\..\\Datasets\\dataset\\training_data\\annotations\\0001438955.json', '..\\..\\Datasets\\dataset\\training_data\\annotations\\0001456787.json', '..\\..\\Datasets\\dataset\\training_data\\annotations\\0001463282.json', '..\\..\\Datasets\\dataset\\traini

  4%|▍         | 6/149 [00:00<00:02, 56.13it/s]

{'box': [292, 91, 376, 175], 'text': 'R&D', 'label': 'other', 'words': [{'box': [292, 91, 376, 175], 'text': 'R&D'}], 'linking': [], 'id': 0}
{'box': [219, 316, 225, 327], 'text': ':', 'label': 'question', 'words': [{'box': [219, 316, 225, 327], 'text': ':'}], 'linking': [], 'id': 1}
{'box': [95, 355, 169, 370], 'text': 'Suggestion:', 'label': 'question', 'words': [{'box': [95, 355, 169, 370], 'text': 'Suggestion:'}], 'linking': [[2, 16]], 'id': 2}
{'box': [482, 268, 518, 282], 'text': 'Date:', 'label': 'question', 'words': [{'box': [482, 268, 518, 282], 'text': 'Date:'}], 'linking': [[3, 12]], 'id': 3}
{'box': [511, 309, 570, 323], 'text': 'Licensee', 'label': 'answer', 'words': [{'box': [511, 309, 570, 323], 'text': 'Licensee'}], 'linking': [[13, 4]], 'id': 4}
{'box': [211, 651, 217, 662], 'text': '', 'label': 'question', 'words': [{'box': [211, 651, 217, 662], 'text': ''}], 'linking': [], 'id': 5}
{'box': [461, 605, 483, 619], 'text': 'Yes', 'label': 'question', 'words': [{'box': [4

 13%|█▎        | 20/149 [00:00<00:02, 58.19it/s]

{'box': [505, 427, 575, 439], 'text': 'COMMENTS', 'label': 'question', 'words': [{'box': [505, 427, 575, 439], 'text': 'COMMENTS'}], 'linking': [[0, 47]], 'id': 0}
{'box': [17, 174, 70, 189], 'text': 'Brand:', 'label': 'question', 'words': [{'box': [17, 174, 70, 189], 'text': 'Brand:'}], 'linking': [[1, 35]], 'id': 1}
{'box': [18, 201, 113, 214], 'text': 'Commercial:', 'label': 'question', 'words': [{'box': [18, 201, 113, 214], 'text': 'Commercial:'}], 'linking': [[2, 38]], 'id': 2}
{'box': [17, 254, 98, 269], 'text': 'Supplier:', 'label': 'question', 'words': [{'box': [17, 254, 98, 269], 'text': 'Supplier:'}], 'linking': [[3, 42]], 'id': 3}
{'box': [442, 198, 498, 211], 'text': 'Sample:', 'label': 'question', 'words': [{'box': [442, 198, 498, 211], 'text': 'Sample:'}], 'linking': [[4, 8]], 'id': 4}
{'box': [20, 362, 86, 377], 'text': 'Chicago:', 'label': 'question', 'words': [{'box': [20, 362, 86, 377], 'text': 'Chicago:'}], 'linking': [[5, 6]], 'id': 5}
{'box': [302, 359, 330, 374], 

...

{'box': [73, 147, 93, 160], 'text': 'TO:', 'label': 'question', 'words': [{'box': [73, 147, 93, 160], 'text': 'TO:'}], 'linking': [[0, 13]], 'id': 0}
{'box': [73, 176, 109, 191], 'text': 'FROM:', 'label': 'question', 'words': [{'box': [73, 176, 109, 191], 'text': 'FROM:'}], 'linking': [[1, 14]], 'id': 1}
{'box': [74, 208, 95, 221], 'text': 'CC:', 'label': 'question', 'words': [{'box': [74, 208, 95, 221], 'text': 'CC:'}], 'linking': [[2, 15]], 'id': 2}
{'box': [142, 649, 219, 664], 'text': 'PACK/CARTON', 'label': 'question', 'words': [{'box': [142, 649, 219, 664], 'text': 'PACK/CARTON'}], 'linking': [[3, 5]], 'id': 3}
{'box': [142, 779, 229, 794], 'text': 'REQUIREMENTS', 'label': 'question', 'words': [{'box': [142, 779, 229, 794], 'text': 'REQUIREMENTS'}], 'linking': [[4, 40]], 'id': 4}
{'box': [380, 649, 412, 664], 'text': 'PACK', 'label': 'answer', 'words': [{'box': [380, 649, 412, 664], 'text': 'PACK'}], 'linking': [[3, 5]], 'id': 5}
{'box': [142, 860, 174, 873], 'text': 'JOB#', 'lab

 92%|█████████▏| 46/50 [00:00<00:00, 54.98it/s]

{'box': [524, 54, 697, 176], 'text': '', 'label': 'other', 'words': [{'box': [524, 54, 697, 176], 'text': ''}], 'linking': [], 'id': 0}
{'box': [680, 672, 700, 773], 'text': '87125460', 'label': 'other', 'words': [{'box': [680, 672, 700, 773], 'text': '87125460'}], 'linking': [], 'id': 1}
{'box': [77, 683, 166, 703], 'text': 'APPROVALS', 'label': 'question', 'words': [{'box': [77, 683, 166, 703], 'text': 'APPROVALS'}], 'linking': [[2, 24], [2, 24]], 'id': 2}
{'box': [80, 730, 122, 747], 'text': 'DATE', 'label': 'question', 'words': [{'box': [80, 730, 122, 747], 'text': 'DATE'}], 'linking': [[3, 24], [3, 24]], 'id': 3}
{'box': [81, 821, 119, 834], 'text': 'DATE', 'label': 'question', 'words': [{'box': [81, 821, 119, 834], 'text': 'DATE'}], 'linking': [[4, 25], [4, 25]], 'id': 4}
{'box': [451, 725, 500, 745], 'text': 'STUDY', 'label': 'question', 'words': [{'box': [451, 725, 500, 745], 'text': 'STUDY'}], 'linking': [], 'id': 5}
{'box': [510, 725, 583, 743], 'text': 'DIRECTOR', 'label': '

100%|██████████| 50/50 [00:00<00:00, 53.31it/s]

{'box': [99, 196, 114, 206], 'text': 'TO:', 'label': 'question', 'words': [{'box': [99, 196, 114, 206], 'text': 'TO:'}], 'linking': [[0, 18]], 'id': 0}
{'box': [156, 242, 184, 253], 'text': 'ATTN:', 'label': 'question', 'words': [{'box': [156, 242, 184, 253], 'text': 'ATTN:'}], 'linking': [[1, 19]], 'id': 1}
{'box': [496, 191, 523, 202], 'text': 'DATE:', 'label': 'question', 'words': [{'box': [496, 191, 523, 202], 'text': 'DATE:'}], 'linking': [[2, 20]], 'id': 2}
{'box': [464, 223, 523, 234], 'text': 'ADVERTISER:', 'label': 'question', 'words': [{'box': [464, 223, 523, 234], 'text': 'ADVERTISER:'}], 'linking': [[3, 5]], 'id': 3}
{'box': [475, 256, 522, 267], 'text': 'PRODUCT:', 'label': 'question', 'words': [{'box': [475, 256, 522, 267], 'text': 'PRODUCT:'}], 'linking': [[4, 6]], 'id': 4}
{'box': [536, 223, 590, 233], 'text': 'LORILLARD', 'label': 'answer', 'words': [{'box': [536, 223, 590, 233], 'text': 'LORILLARD'}], 'linking': [[3, 5]], 'id': 5}
{'box': [538, 256, 585, 267], 'text':